## BECS 2 Data Challenge: Predicting Cancer Status of Patients
Koch Kilian (ZHAW), Rieder Jonathan (ZHAW), Yar Kevin (ZHAW)

The aim of this project work is to predict 6 different healt status of patients (healthy, breast-, lung-, pancreatic-, colorectal- and prostate cancer). The prediction is based on mass spectroscopy data of different protein levels meassured in patients blood. 

## Part 1: Loading packages and define initial Parameters

In [1]:
#import necessary packages
import os
import random
import xgboost
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn import metrics
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SequentialFeatureSelector 
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

os.getcwd()
"""
usage joblib
# save the model: 
joblib.dump(model , "model.pkl")
# load the model:
model = joblib.load("model.pkl")
"""

'\nusage joblib\n# save the model: \njoblib.dump(model , "model.pkl")\n# load the model:\nmodel = joblib.load("model.pkl")\n'

## Part 2: Data Preprocessing
In this section the raw data is preprocessed

In [ ]:
#Kilian insert your code here

## Part 3: Data Preparation
In this section the raw data is prepared and reshaped to be fed into the different models. Furhtermore, the distribution of the input data is visualized to check if the data set is balanced. The data is converted into two main variables X (patiens and the coresponding protein quantities) and y (patients and the coresponding healt condition).


In [ ]:
#create pandas dataframe
path = "tidy.csv"
pathMet = "metadata.csv"
tidy = pd.read_csv(path, sep=",")
tidyMet = pd.read_csv(pathMet, sep=";", index_col=0)

#remove samples which are not in the metadata index column (quality controle etc)
tidy = tidy[ (tidy["R.FileName"].isin(tidyMet.index)) ]
tidyMer    = pd.merge(tidy, tidyMet, how="left", on="R.FileName")
tidySub = tidyMer[["R.FileName", "uniprot", "meanAbu", "Cancer"]]
tidySub.Cancer.value_counts()

In [ ]:
#reshape data for model
#X data
tidyReshaped = tidySub.pivot(index = "R.FileName", columns = "uniprot", values = "meanAbu")
tidyReshaped.head()

#y condition
Group =  tidySub.drop(["uniprot", "meanAbu"], axis=1)
Group = Group.drop_duplicates().reset_index(drop=True)
Group.head()

#merge X and y and set dataframe to numerical values
data = pd.merge(tidyReshaped, Group, how="left", on="R.FileName")
data = data.set_index("R.FileName")

X_ = data.iloc[:, :-1].apply(np.log2)
y_ = data.iloc[:,-1]

#check first 10 entries of the dataframe 
data[:10]

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(30, 5))

ax.boxplot(X_)

### Part 4: Creation of Data Sets, Feature and Model Selection
* Create pipeline for imputing, scaling !! **Scaling is not needed for Random Forest**
* (https://towardsdatascience.com/how-data-normalization-affects-your-random-forest-algorithm-fbc6753b4ddf)
* Creation of training, validation and test sets
* Feature Selection, Engineering
* Model Selection

In [ ]:
#Make pipeline
dataPrepPipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ]) 

#X is already purely numerical
X = dataPrepPipe.fit_transform(X_)

In [ ]:
# Encode target labels
labEnc = preprocessing.LabelEncoder() 
y = labEnc.fit_transform(y_) 
set(zip(y_, y))

In [ ]:
# Save X and y for next session
joblib.dump(y, "Models/y.pkl")
joblib.dump(X, "Models/X.pkl")


In [ ]:
# load saved X and y 
y = joblib.load("Models/y.pkl")
X = joblib.load("Models/X.pkl")


## Feature selection

### Two possibilities:
* Tree-based feature selection - sklearn.feature_selection.SelectFromModel
    * May be used with sklearn.tree models
##
* Sequential Feature Selection - sklearn.feature_selection.SequentialFeatureSelector
    * May be used with xgboost

In [ ]:
# Tree Feature Selection 

"""
Can be used in pipeline
clf = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l1"))),
  ('classification', RandomForestClassifier())
])
"""
#create Random Forest classifier with default hyperparameters
raFo = RandomForestClassifier(random_state=1)
raFo = raFo.fit(X, y)

#checkout importance in a histogram
plt.hist(raFo.feature_importances_, bins=100)
plt.title("Histogram of the feature importance for all 2730 proteins")
plt.xlabel("Importance")
plt.ylabel("Count")


#get the reduced X
model = SelectFromModel(estimator = raFo, prefit=True)
X_new = model.transform(X)

print(f"Original X shape: {X.shape}")
print(f"Feature selected X_new shape: {X_new.shape}")


In [ ]:
# sequential Feature Selection 

params = dict(tree_method="exact", 
                eval_metric='mlogloss',
                use_label_encoder =False)

clf_XGRF = xgboost.XGBClassifier(random_state=7, **params)
model = SequentialFeatureSelector(estimator = clf_XGRF, n_features_to_select = 0.20, cv = 10,  n_jobs=-1)
model.fit(X,y)
X_new = model.transform(X)
#checkout importance in a histogram
plt.hist(raFo.feature_importances_, bins=100)

print(f"Original X shape: {X.shape}")
print(f"Feature selected X_new shape: {X_new.shape}")

In [ ]:
# Visualize feature importance

importances = raFo.feature_importances_

std = np.std([tree.feature_importances_ for tree in raFo.estimators_], axis=0)
std.sort()

forest_importances = pd.Series(importances)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
ax.get_xaxis().set_visible(False)
fig.tight_layout()
plt.show()


## Part 5: Model Training


In [ ]:
# import already trained model

clf_RF = joblib.load("Models/clf_RF_X_new.pkl")
clf_XGRF = joblib.load("Models/clf_XGRF_X_new.pkl")

### Random Forest

In [ ]:
#split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=4)
clf_RF = RandomForestClassifier(random_state=1)
clf_RF.fit(X_train ,y_train)
y_RFpred = clf_RF.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy Random Forest:",metrics.accuracy_score(y_test , y_RFpred))
print(classification_report(y_test, y_RFpred))

### XGBoost Random Forest

In [ ]:
#define some parameters for xgboost to avoid warnings
params = dict(tree_method="exact", 
                eval_metric='mlogloss',
                use_label_encoder =False)

clf_XGRF = xgboost.XGBClassifier(random_state=7, **params)

clf_XGRF.fit(X_train ,y_train)
y_XGRFpred = clf_XGRF.predict(X_test)

print("Accuracy XGBoost Random Forest:",metrics.accuracy_score(y_test , y_XGRFpred))
print(classification_report(y_test, y_XGRFpred))

In [ ]:
#save the model
joblib.dump(clf_RF, "Models/clf_RF_X_new.pkl")
joblib.dump(clf_XGRF, "Models/clf_XGRF_X_new.pkl")

## Part 6: Hyperparameter Tuning
In this chapter the model is optimized by hyperparameter tuning. A random grid search is applied to selected hyperparameters of both models. The hyperparametertuning follows the instructions of the following publication https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74 (31.12.21).

In [ ]:
#Hyperparameter tuning for the random forest classifier using random grid search
#get model parameters(delete after hyperparameter tuning)
#params = clf_RF.get_params()
#params

#Define hyperparameters for tuning
n_estimators_RF = [int(x) for x in np.linspace(start=200, stop=2000, num=10)] #number of trees
max_features_RF = ['auot', 'sqrt'] #number of features
max_depth_RF = [int(x) for x in np.linspace(10, 110, num=1)] #numbber of levels
max_depth_RF.append(None)
min_samples_split_RF = [2, 5, 10] #minimum number of samples required to split a note
min_samples_leaf_RF = [1, 2, 4] #minimum number of samples required at each leaf node
bootstrap_RF = [True, False] #method of selecting samples for training

#initialize random grid
random_grid = {'n_estimators': n_estimators_RF,
            'max_features': max_features_RF,
            'max_depth': max_depth_RF,
            'min_samples_split': min_samples_split_RF,
            'min_samples_leaf': min_samples_leaf_RF,
            'bootstrap': bootstrap_RF}

#define model parameters for random grid search
RF_random = RandomizedSearchCV(estimator = clf_RF,
            param_distributions=random_grid, n_iter=200,
            cv=5, verbose=2, random_state=13, n_jobs=-1)

#fit the random search model
RF_random.fit(X_train, y_train)

#get best hyperparameters from the model
RF_random.best_params_

In [ ]:
#run model with optimized hyperparameters
clf_RF_tuned = RandomForestClassifier(n_estimators=400,
                                    min_samples_split=2,
                                    min_samples_leaf=4,
                                    max_features='sqrt',
                                    max_depth= None,
                                    bootstrap= True,
                                    random_state=1)
clf_RF_tuned.fit(X_train ,y_train)
y_RFpred_tuned = clf_RF_tuned.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy Random Forest:",metrics.accuracy_score(y_test , y_RFpred_tuned))

In [ ]:
params = clf_RF.get_params()
params

In [ ]:
#Hyperparameter tuning for the random forest classifier using random grid search
#get model parameters(delete after hyperparameter tuning)
params = clf_XGRF.get_params()
params

In [ ]:
clf_RF = RandomForestClassifier()
# second try hyperparameter tuning

#Hyperparameter tuning for the random forest classifier using random grid search
#get model parameters(delete after hyperparameter tuning)
#params = clf_RF.get_params()
#params\n,

#Define hyperparameters for tuning\n,
n_estimators_RF = [x for x in np.linspace(start=50, stop=500, num=10, dtype=int)] #number of trees
criterion_RF    = ["gini", "entropy"]

max_depth_RF = [int(x) for x in np.arange(1, 20)] #numbber of levels
max_depth_RF.append(None)


min_samples_split_RF = [1, 2, 5, 10] #minimum number of samples required to split a note
min_samples_leaf_RF = [np.arange(start=1, stop=5)] #minimum number of samples required at each leaf node

max_features_RF = ['auto', 'sqrt', "log2"] #number of features

class_weight        = []  #<---     SUPER WICHTIG FÜR BIOMARKER

bootstrap_RF = [True, False] #method of selecting samples for training
#initialize random grid \n,
# RANDOM FOREST PARAMS\n,
random_grid = {'n_estimators'   : n_estimators_RF,
            'max_features'      : max_features_RF,
            'max_depth'         : max_depth_RF,                        
            'min_samples_split' : min_samples_split_RF,
            'bootstrap'         : bootstrap_RF}

#define model parameters for random grid search
RF_random = RandomizedSearchCV(estimator = clf_RF,
            param_distributions=random_grid, n_iter=100,
            cv=5, verbose=0, n_jobs=-1)

#fit the random search model
RF_random.fit(X_train, y_train)

#get best hyperparameters from the model
RF_random.best_params



params = {'n_estimators': 350,
    'min_samples_split': 2,
    'max_features': 'sqrt',
    'max_depth': None,
    'bootstrap': False}

clf_RF = RandomForestClassifier(**params)
clf_RF.fit(X_train, y_train) 

y_RFpred = clf_RF.predict(X_test)

# Model Accuracy, how often is the classifier correct?\n,
print("Accuracy Random Forest: ",metrics.accuracy_score(y_test , y_RFpred))
print(classification_report(y_test, y_RFpred))

## Part 7: Model Validation

In [ ]:
# visualize confusion matrix


cm = confusion_matrix(y_test, y_RFpred, labels=clf_RF.classes_) # calculate value
disp = ConfusionMatrixDisplay(confusion_matrix=cm,              # display
                              display_labels=clf_RF.classes_)
disp.plot(); 



## Compare uncertainty of Data and Model

In [ ]:
scores = cross_val_score(clf_RF, X_new, y, cv=5, scoring='accuracy')
Udata = scores.std()

modAcuRF = []
for rs in range(1,6):
    model = RandomForestClassifier(random_state=random.randrange(rs))
    model.fit(X_train, y_train)
    modAcuRF += [accuracy_score(y_test, model.predict(X_test))]

Umodel = np.std(modAcuRF)

print("Uncertainty in the data: %.3f" % Udata)
print("Uncertainty in the model: %.3f" % Umodel)
print("The model performance is %.3f ± %.3f ± %.3f" % (scores.mean(),Udata,Umodel))

In [ ]:
scores = cross_val_score(clf_XGRF, X_new, y, cv=5, scoring='accuracy')
Udata = scores.std()

modAcuXGRF = []
for rs in range(1,6):
    model = xgboost.XGBClassifier(random_state=random.randrange(rs), **params)
    model.fit(X_train, y_train)
    modAcuXGRF += [accuracy_score(y_test, model.predict(X_test))]

Umodel = np.std(modAcuXGRF)

print("Uncertainty in the data: %.3f" % Udata)
print("Uncertainty in the model: %.3f" % Umodel)
print("The model performance is %.3f ± %.3f ± %.3f" % (scores.mean(),Udata,Umodel))